In [1]:
import math
import torch
import torch.nn as nn
from fairseq.models.bart import BARTModel

In [2]:
bart = BARTModel.from_pretrained('/home/ml/cadencao/Downloads/BART_models/bart.large.xsum',
                                 checkpoint_file='model.pt',
                                 data_name_or_path='/home/ml/cadencao/Downloads/BART_models/bart.large.xsum')

In [3]:
bart.cuda()
bart.eval()
bart.half()
print('- activate evaluation mode')

- activate evaluation mode


In [4]:
encode_func = bart.encode
decode_func = bart.decode

In [5]:
bart_encoder = bart.model.encoder
bart_decoder = bart.model.decoder
print(type(bart.model))
print(type(bart_encoder))
print(type(bart_decoder))

<class 'fairseq.models.bart.model.BARTModel'>
<class 'fairseq.models.transformer.TransformerEncoder'>
<class 'fairseq.models.transformer.TransformerDecoder'>


#### Read XSum

In [6]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [7]:
document_path = '/home/ml/cadencao/XSum/fairseq_files/test.source'
target_path = '/home/ml/cadencao/XSum/fairseq_files/test.target'
xsum_source = read_lines(document_path)
xsum_target = read_lines(target_path)
print(len(xsum_source))
assert len(xsum_source) == len(xsum_target)

11301


#### Tokenization

In [8]:
import spacy
import rouge

from fairseq.data.data_utils import collate_tokens

In [9]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

aggregator = 'Avg'
apply_avg = aggregator == 'Avg'
apply_best = aggregator == 'Best'
print('Evaluation with {}'.format(aggregator))

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=apply_avg,
                        apply_best=apply_best,
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=True)

Evaluation with Avg


In [10]:
nlp = spacy.load('en')

In [11]:
INDEX = 1819
test_inputs = [xsum_source[INDEX]]
src_tokens = collate_tokens([encode_func(i) for i in test_inputs], pad_idx=1, left_pad=True)
src_tokens = src_tokens.cuda()
print(src_tokens.shape)

src_lengths = torch.sum(src_tokens != 1, dim=1)
print(src_lengths)

torch.Size([1, 589])
tensor([589], device='cuda:0')


In [12]:
# encoding

In [13]:
encoder_out = bart_encoder(src_tokens, src_lengths=src_lengths)
print(encoder_out.encoder_out.shape)  # [seq_len, batch_size, hidden_size]

torch.Size([589, 1, 1024])


In [14]:
# decoding

In [15]:
init_input = torch.tensor([[2, 0]] * src_tokens.shape[0], dtype=torch.long).cuda()
min_decode_step, max_decode_step, pad_id, eos_id = 10, 50, 1, 2
softmax = nn.Softmax(dim=1)

for step in range(max_decode_step):
    decoder_outputs = bart_decoder(init_input, encoder_out, features_only=False)
    logits = decoder_outputs[0][:, -1, :]  # [batch_size, vocab]
    probs = softmax(logits)
    assert logits.shape == probs.shape
    attn = decoder_outputs[1]['attn'][0]  # [batch_size, prev_token_len, src_token_len]
    assert logits.dim() == 2 and attn.dim() == 3
    logits[:, pad_id] = -math.inf  # never select pad
    probs[:, pad_id] = 0.0  # never select pad

    if step + 1 < min_decode_step:
        logits[:, eos_id] = -math.inf

    # preds = torch.argmax(logits, dim=1)
    value, indices = torch.topk(probs, 5, dim=1)
    selected_token = indices[:, 0]

    if step == 15:
        print(indices)
        selected_token = indices[:, 1]
    elif step == 19:
        selected_token = indices[:, 0]
    elif step == 19:
        selected_token = indices[:, 0]

    init_input = torch.cat([init_input, selected_token.unsqueeze(1)], dim=-1)
    print("- {:02d}: {} ({:.2f})".format(step, decode_func(selected_token), probs.squeeze()[selected_token.item()]), end='\n')

    if selected_token.item() == eos_id:
        break

print(attn[0].shape)

- 00: The (0.47)
- 01:  brother (0.78)
- 02:  of (0.92)
- 03:  a (0.59)
- 04:  victim (0.47)
- 05:  of (0.92)
- 06:  Bloody (0.73)
- 07:  Sunday (0.93)
- 08:  has (0.88)
- 09:  said (0.19)
- 10:  a (0.39)
- 11:  planned (0.32)
- 12:  march (0.57)
- 13:  by (0.86)
- 14:  veterans (0.32)
tensor([[  7,  11,   9, 333, 108]], device='cuda:0')
- 15:  in (0.12)
- 16:  L (0.69)
- 17: ond (0.86)
- 18: ond (0.92)
- 19: erry (0.96)
- 20:  is (0.46)
- 21:  an (0.40)
- 22:  " (0.71)
- 23: act (0.70)
- 24:  of (0.92)
- 25:  pure (0.89)
- 26:  provocation (0.96)
- 27: ". (0.69)
- 28:  (0.91)
torch.Size([30, 589])


In [16]:
init_input

tensor([[    2,     0,   133,  2138,     9,    10,  1802,     9, 38274,   395,
            34,    26,    10,  1904,  6674,    30,  4823,    11,   226,  2832,
          2832, 11228,    16,    41,    22,  7257,     9,  8309, 29995,   845,
             2]], device='cuda:0')

In [17]:
decode_func(torch.tensor([0,   133,  2138,     9,    65,     9,     5,  1680,     9, 38274,   395,    34,    26,    10,  1904,  6674,    30,   320,  3878,          11,   226,  2832,  2832, 11228,    16,    41,    22,  7257,     9,         8309, 29995,   845,     2]))

'The brother of one of the victims of Bloody Sunday has said a planned march by former soldiers in Londonderry is an "act of pure provocation".'

In [18]:
decode_func(torch.tensor([ 0, 133, 2138, 9, 65, 9, 5, 1680, 9, 38274,   395,    34,    26,    10,  1904,  6674,    30,   320,  3878, 11,   226,  2832,  2832, 11228,    16,    22,   260,  1760,     9, 8309, 29995,   845,     2]))

'The brother of one of the victims of Bloody Sunday has said a planned march by former soldiers in Londonderry is "an act of pure provocation".'

In [19]:
decode_func(torch.tensor([0,   133,  2138,     9,    65,     9,     5,  1680,     9, 38274,   395,    34,    26,    10,  1904,  6674,    30,   320,  3878, 11,   226,  2832,  2832, 11228,    16,    41,    22,  7257,     9, 8309, 29995,   113, 2]))

'The brother of one of the victims of Bloody Sunday has said a planned march by former soldiers in Londonderry is an "act of pure provocation"'

In [20]:
hypothesis = decode_func(init_input[0])
print(hypothesis)

The brother of a victim of Bloody Sunday has said a planned march by veterans in Londonderry is an "act of pure provocation".


In [21]:
xsum_target[INDEX]

'Relatives of Bloody Sunday victims have called a march by military veterans in Londonderry "an act of pure provocation".'

In [22]:
all_hypothesis = [hypothesis]
all_references = [xsum_target[INDEX]]

scores = evaluator.get_scores(all_hypothesis, all_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 65.22	R: 78.95	F1: 71.43
	rouge-2:	P: 40.91	R: 50.00	F1: 45.00
	rouge-3:	P: 23.81	R: 29.41	F1: 26.32
	rouge-4:	P: 10.00	R: 12.50	F1: 11.11
	rouge-l:	P: 66.12	R: 77.53	F1: 71.37
	rouge-w:	P: 51.90	R: 34.86	F1: 41.71


In [23]:
out = nlp(decode_func(init_input[0]))
print(out)
print(out.ents)

The brother of a victim of Bloody Sunday has said a planned march by veterans in Londonderry is an "act of pure provocation".
(Bloody Sunday, Londonderry)


In [24]:
"The body of a man whose body was found at the site of the Swansea Bay Power Station collapse has been removed from the site."

'The body of a man whose body was found at the site of the Swansea Bay Power Station collapse has been removed from the site.'

In [25]:
"The body of a man who died when the collapsed Swansea Power Station building collapsed has been removed from the site."

'The body of a man who died when the collapsed Swansea Power Station building collapsed has been removed from the site.'

In [26]:
xsum_source[6220]

'Christopher Huxtable, 34, from Swansea, had been missing since the collapse in February. His body was found on Wednesday and workers who carried out the search formed a guard of honour as it was driven from the site in the early hours of the morning. Ken Cresswell, 57, and John Shaw, 61, both from Rotherham, remain missing. The body of a fourth man, Michael Collings, 53, from Brotton, Teesside, was previously recovered from the site. Swansea East MP Carolyn Harris, who has been involved with the family since the incident, said they still did not know all the facts about the collapse. She said: "I feel very sad. My heart and my prayers go out to the family who have waited desperately for Christopher\'s body to be found. They can finally have closure, and say goodbye to him and grieve his loss. "But let\'s not forget that there\'s two other families who are still waiting for their loved ones to be returned." The building was due for demolition when it partially collapsed in February.'

In [27]:
# 15338,  3864,  9368

In [28]:
decode_func(torch.tensor([9368]))

' tower'